# day 12

https://adventofcode.com/2021/day/12

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day12.txt')

LOGGER = logging.getLogger('day12')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """start-A
start-b
A-c
A-b
b-d
A-end
b-end"""

test_data_2 = """dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc"""

test_data_3 = """fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
import itertools
import string
import networkx as nx

def parse_data(d, dupe_nodes=False):
    g = nx.Graph()
    for pair in d.split('\n'):
        a, b = pair.split('-')
        g.add_edge(a, b)
    
    if dupe_nodes is True:
        # now duplicate every small node
        lowercase_nodes = [n for n in g.nodes if n.lower() == n and n not in ['start', 'end']]
        for lowercase_node in lowercase_nodes:
            new_node = f"{lowercase_node}2"
            for neighbor in g.neighbors(lowercase_node):
                g.add_edge(new_node, neighbor)
        
    g2 = nx.MultiGraph(g)
    
    # for start, end, and lowercase, nodes and regular lines
    # uppercase --> new edges
    cap_nodes = [n for n in g2.nodes if n.upper() == n]
    
    while len(cap_nodes) > 0:
        cap_node_now = cap_nodes.pop()
        cap_node_now_neighbors = set(g2.neighbors(cap_node_now))
        for (a, b) in itertools.combinations(cap_node_now_neighbors, 2):
            g2.add_edge(a, b)
        
        g2.remove_node(cap_node_now)
    
    return g2

In [ ]:
g = parse_data(test_data)
nx.draw_networkx(g, with_labels=True)

In [ ]:
g = parse_data(test_data, dupe_nodes=True)
nx.draw_networkx(g, with_labels=True)

#### function def

In [ ]:
from networkx.algorithms.simple_paths import all_simple_paths

def q_1(data):
    g = parse_data(data)    
    return sum(1 for path in all_simple_paths(g, 'start', 'end'))

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 10
    assert q_1(test_data_2) == 19
    assert q_1(test_data_3) == 226
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def is_ordered(path):
    for (i, node) in enumerate(path):
        if node[-1] == '2' and node[:-1] not in path[:i]:
            return False
    return True

assert is_ordered(['start', 'b', 'd', 'b2', 'end'])
assert not is_ordered(['start', 'b2', 'd', 'b', 'end'])
assert is_ordered(['start', 'a' , 'end'])
assert not is_ordered(['start', 'a2' , 'end'])

In [ ]:
def q_2(data):
    g = parse_data(data, dupe_nodes=True)
    return [path for path in all_simple_paths(g, 'start', 'end')
            if is_ordered(path)]
    return sum(1 for path in all_simple_paths(g, 'start', 'end')
               if is_ordered(path))

In [ ]:
sorted(q_2(test_data))

In [ ]:
# g = q_2(test_data)

In [ ]:
# nx.draw_networkx(g, with_labels=True)

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 36
    assert q_2(test_data_2) == 103
    assert q_2(test_data_3) == 3509
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin

In [ ]:
from collections import defaultdict
neighbours = defaultdict(list)

for line in load_data().split('\n'):
    a, b = line.strip().split('-')
    neighbours[a] += [b]
    neighbours[b] += [a]

def search(part, seen=set(), cave='start'):
    if cave == 'end': return 1
    if cave in seen:
        if cave == 'start': return 0
        if cave.islower():
            if part == 1: return 0
            else: part = 1

    return sum(search(part, seen|{cave}, n)
                 for n in neighbours[cave])

print(search(part=1), search(part=2))